In [41]:
import os
import re
import openai
from dotenv import load_dotenv

from langchain.chat_models import ChatOpenAI
from langchain.prompts import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

In [42]:
load_dotenv()

True

In [43]:
# Secret keys from .env file
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [100]:
# Directory Variables
CUR_DIR = os.path.dirname(os.path.abspath('__file__'))
APP_DIR = os.path.abspath(os.path.join(CUR_DIR, os.pardir))
# LOG_FILE = ("twitter-bot.log")
RAW_TEXT_FILE = r"C:\Users\manas\Coding Playground\PythonP\TwitterBot\content_ideas.txt"
# TEXT_FILE = os.path.join(APP_DIR, "data/processed/quants_tweets.txt")

# print variables
# print(f"Application Dir: \n\t{APP_DIR}")
print(f"Raw Tweet File Dir: \n\t{RAW_TEXT_FILE}")
# print(f"Processed Tweet File Dir: \n\t{TEXT_FILE}")

Raw Tweet File Dir: 
	C:\Users\manas\Coding Playground\PythonP\TwitterBot\content_ideas.txt


In [93]:
## Prompt Engineering with ChatGPT

In [94]:
llm = ChatOpenAI(temperature=0.3,
                 openai_api_key=OPENAI_API_KEY,
                 model_name='gpt-3.5-turbo-0613',
                )

In [95]:
def generate_response(
    llm: ChatOpenAI, coding_topic: str, coding_explaination: str
) -> tuple[str, str]:
    """Generate AI Twitter Content for CodingAunty Twitter Account

    Parameters:
        - llm:  pre-trained ChatOpenAi large language model
        - coding_topic: Topic related to Python
        - coding_explaination: Explanation of the coding topic

    Returns:
        - tuple[long response,short reposonse]: Chat GPT long and short responses
    """
    # System Template for LLM to follow
    system_template = """
        You are an incredibly wise and smart coder and programmer that lives and breathes the world of Python Programming Language.
        Your goal is to writing short-form content for twitter given a `topic` in the area of Python programming  and a `explaination` from the user.

        % RESPONSE TONE:

        - Your response should be given in an active voice and be opinionated
        - Your tone should be serious w/ a hint of wit and sarcasm
        
        % RESPONSE FORMAT:
        
        - Be extremely clear and concise
        - Respond with phrases no longer than two sentences
        - Do not respond with emojis
        
        % RESPONSE CONTENT:

        - Include specific examples of where this is used in the programming space
        - If you don't have an answer, say, "Sorry, I'll have to ask the Python Programming Gods!"    

        
        % RESPONSE TEMPLATE:

        - Here is the response structure: 
            Tweet: A captivating engaging tweet, hooking and engaging the user
            Hash Tags: 3 relevant hashtags
    
    """
    # system prompt template to follow
    system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)

    # human template for input
    human_template = "topic to write about is {topic}, and the exolaination will be {explaination}. Keep the total response under 200 words total! No emojis should exist in the tweet or hash tags"
    human_message_prompt = HumanMessagePromptTemplate.from_template(
        human_template, input_variables=["topic", "explaination"]
    )

    # chat prompt template construction
    chat_prompt = ChatPromptTemplate.from_messages(
        [system_message_prompt, human_message_prompt]
    )

    # get a completed chat using formatted template with topic and title
    final_prompt = chat_prompt.format_prompt(
        topic=coding_topic, explaination=coding_explaination
    ).to_messages()

    # pass template through llm and extract content attribute
    first_response = llm(final_prompt).content

    # construct AI template, to pass back OpenAI response
    ai_message_prompt = AIMessagePromptTemplate.from_template(first_response)

     # additional prompt to remind ChatGPT of length requirement
    reminder_template = "This was good, but find and remove any emojis or icons. Make it more engaging and informative. Add actionable insights. "
    reminder_prompt = HumanMessagePromptTemplate.from_template(reminder_template)

    # chat prompt template construction with additional AI response and length reminder
    chat_prompt2 = ChatPromptTemplate.from_messages(
        [system_message_prompt, human_template, ai_message_prompt, reminder_prompt]
    )

    # get a completed chat using formatted template with topic and title
    final_prompt = chat_prompt2.format_prompt(
        topic=coding_topic, explaination=coding_explaination
    ).to_messages()

    # pass template through llm and extract content attribute
    short_response = llm(final_prompt).content

    return first_response, short_response



In [96]:
# Function to generate multiple unique tweets for each topic
def generate_unique_tweets(coding_topic, coding_explaination, num_tweets=3):
        generated_tweets = set()

        while len(generated_tweets) < num_tweets:
            long_response, short_response = generate_response(llm, coding_topic, coding_explaination)

             # Check for uniqueness
            # if long_response not in generated_tweets:
            #     generated_tweets.add(long_response)
            #     print(f"Tweet: {long_response}")
            
            # Check for uniqueness
            if short_response not in generated_tweets:
                generated_tweets.add(short_response)
                print(f"Tweet: {short_response}")

In [97]:
generate_unique_tweets(coding_topic='List Comprehensions', 
    coding_explaination='How to use list comprehensions for more concise and readable code.')

Tweet: Tweet: "Say goodbye to long and tedious loops! 🚫✋ Level up your Python code with list comprehensions! 🚀 They let you create lists in a concise and readable way, making your code more efficient and elegant. 💡 #PythonProgramming #CodeOptimization #ReadableCode"

Hash Tags: #PythonProgramming #CodeOptimization #ReadableCode
Tweet: Tweet: "Say goodbye to lengthy loops! 🚫✋ List comprehensions in Python are a game-changer for concise and readable code. Transform and filter lists effortlessly. 🔄 Boost your coding efficiency today! 💪 #Python #CodeSimplicity #EfficiencyTips"

Hash Tags: #Python #CodeSimplicity #EfficiencyTips
Tweet: Tweet: "Boost your Python code's efficiency with list comprehensions! 🚀 Say goodbye to lengthy loops and hello to concise, readable code. 📝 Create lists effortlessly and eliminate repetitive code. 💪 Level up your programming skills! #Python #ListComprehensions #CodeEfficiency"

Hash Tags: #Python #ListComprehensions #CodeEfficiency


In [98]:
## Preprocessing content-ideas from text file

In [99]:
# Take text file in table structure from GPT and process it by line divisors (|). 
# Then create dicitonary and store in desired format back to text file.

In [104]:
file = open(RAW_TEXT_FILE, "r")
python_tweets = {}

for line_no, line in enumerate(file.readlines()):
    # start 2nd row to avoid heading and underlines
    if line_no > 1:
        # split on line divisors
        items = line.split('|')
        # capture and ensure int and str formatting
        tweet_no = int(items[1])
        coding_topic = items[2].strip()
        coding_explaination = items[3].strip().strip('"')
        # store within dict
        python_tweets[tweet_no] = {}
        python_tweets[tweet_no]['topic'] = coding_topic
        python_tweets[tweet_no]['explaination'] = coding_explaination
        # print tweet no, topic and title
        print(f"{tweet_no}_{coding_topic}_{coding_explaination}")
file.close()

1_List Comprehensions_How to use list comprehensions for more concise and readable code.
2_Pythonic Loops_Tips on writing Pythonic loops and iterating over sequences.
3_Effective use of `lambda`_Demonstrating the power and convenience of lambda functions.
4_Understanding Decorators_Simplifying the concept of decorators and their usage in Python.
5_Error Handling Best Practices_How to effectively use try-except blocks for error management.
6_Generators Explained_Introducing generators and how they differ from regular functions.
7_Efficient File Reading_Best practices for reading files in Python without using excessive memory.
8_Regular Expressions_Simplifying regex in Python for string searching and manipulation.
9_Virtual Environments_Why and how to use virtual environments in Python.
10_Unpacking Sequences_Tips on unpacking sequences into variables effectively.
11_The Power of `enumerate`_Using `enumerate` for more Pythonic loops.
12_Dictionary Comprehensions_Creating dictionaries eff

IndexError: list index out of range

In [106]:
python_tweets

{1: {'topic': 'List Comprehensions',
  'explaination': 'How to use list comprehensions for more concise and readable code.'},
 2: {'topic': 'Pythonic Loops',
  'explaination': 'Tips on writing Pythonic loops and iterating over sequences.'},
 3: {'topic': 'Effective use of `lambda`',
  'explaination': 'Demonstrating the power and convenience of lambda functions.'},
 4: {'topic': 'Understanding Decorators',
  'explaination': 'Simplifying the concept of decorators and their usage in Python.'},
 5: {'topic': 'Error Handling Best Practices',
  'explaination': 'How to effectively use try-except blocks for error management.'},
 6: {'topic': 'Generators Explained',
  'explaination': 'Introducing generators and how they differ from regular functions.'},
 7: {'topic': 'Efficient File Reading',
  'explaination': 'Best practices for reading files in Python without using excessive memory.'},
 8: {'topic': 'Regular Expressions',
  'explaination': 'Simplifying regex in Python for string searching and

In [107]:
# storing in desired format
# could directly place processed data location here: TEXT_FILE instead of 'python_tweets.txt'
with open('python_tweets.txt', 'w') as f:
    for tweet_no, tweet_info in python_tweets.items():
        #Attaching 3 flags at the end which are currently all False
        tweet_repr = str(tweet_no)+'|'+tweet_info['topic']+'|'+tweet_info['explaination']+'\n'
        f.write(tweet_repr)